In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(appName="Assignment1")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/29 17:44:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#hdfs dfs -mkdir -p data
#hdfs dfs -put data/small_conditions.csv data/

In [3]:
SUPPORT_THRESHOLD = 100
data = sc.textFile("data/small_conditions.csv")
header = data.first() #extract header

In [ ]:
# START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
# PATIENT is the patient identifier
# CODE is a condition identifier 
# DESCRIPTION is the name of the condition

In [16]:
# Apriori Phase 1

# remove header and split by the comma
items = data.filter(lambda row : row != header)\
        .flatMap(lambda line: line.split(","))
# Mapping -> create pairs (item, 1)
itemPairs = items.map(lambda item: (item, 1))
# Reducing
itemCounts = itemPairs.reduceByKey(lambda a, b: a + b)
# Keeping only the ones above the support threshold
freqItemCounts = itemCounts.filter(lambda item: item[1] >= SUPPORT_THRESHOLD)
#items.take(12)
itemCounts.take(10)

[('2017-01-14', 2),
 ('Otitis media', 48),
 ('2012-09-16', 2),
 ('241929008', 8),
 ('2018-06-17', 1),
 ('2018-06-24', 2),
 ('2019-08-02', 1),
 ('667a94d9-6aa1-4b6f-bc38-8ec8a111d946', 1),
 ('33737001', 2),
 ('Fracture of rib', 2)]

In [13]:
# flatMap fica no formato fica numa lista única, pode não dar jeito para o A-priori por causa dos baskets
#### Ver isto melhor

In [5]:
itemCounts.takeOrdered(10, key=lambda x: -x[1]) #Most frequent items

[('', 783),
 ('444814009', 320),
 ('Viral sinusitis (disorder)', 320),
 ('195662009', 193),
 ('Acute viral pharyngitis (disorder)', 193),
 ('10509002', 143),
 ('Acute bronchitis (disorder)', 143),
 ('72892002', 142),
 ('Normal pregnancy', 142),
 ('Body mass index 30+ - obesity (finding)', 90)]

In [6]:
freqItemCounts.collect()

[('10509002', 143),
 ('', 783),
 ('195662009', 193),
 ('Acute viral pharyngitis (disorder)', 193),
 ('444814009', 320),
 ('Viral sinusitis (disorder)', 320),
 ('Acute bronchitis (disorder)', 143),
 ('72892002', 142),
 ('Normal pregnancy', 142)]

In [21]:
# Phase 2 ...